In [ ]:

import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import glob
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
tf.compat.v1.disable_eager_execution()

# Caminhos dos modelos
FACE_MODEL_PATH = 'flavio/models/weights_face_v1.0.0.pb'
PLATE_MODEL_PATH = 'flavio/models/weights_plate_v1.0.0.pb'

# Diretórios
INPUT_BASE_FOLDER = "/mnt/cadastro/br/sc/balneario_camboriu/fotos_veiculares/panoramica/PM_BAL_CAMB_20230920__ORIGINAL"
OUTPUT_BASE_FOLDER = "/mnt/data/flavio"
PROCESSED_LOG = os.path.join(OUTPUT_BASE_FOLDER, 'imagens_processadas.txt')

# Parâmetros
FACE_THRESHOLD = 0.5
PLATE_THRESHOLD = 0.5
OBFUSCATION_KERNEL = (251, 251)
JPEG_QUALITY = 75
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff')

def load_frozen_graph(pb_file_path):
    if not os.path.exists(pb_file_path):
        logging.error(f"Model file not found: {pb_file_path}")
        raise FileNotFoundError(f"Model file not found: {pb_file_path}")
    with tf.io.gfile.GFile(pb_file_path, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    graph = tf.Graph()
    with graph.as_default():
        tf.import_graph_def(graph_def, name="")
    return graph

face_graph = load_frozen_graph(FACE_MODEL_PATH)
face_sess = tf.compat.v1.Session(graph=face_graph)
plate_graph = load_frozen_graph(PLATE_MODEL_PATH)
plate_sess = tf.compat.v1.Session(graph=plate_graph)

# Tensores
face_input_tensor = face_graph.get_tensor_by_name("image_tensor:0")
face_boxes_tensor = face_graph.get_tensor_by_name("detection_boxes:0")
face_scores_tensor = face_graph.get_tensor_by_name("detection_scores:0")
plate_input_tensor = plate_graph.get_tensor_by_name("image_tensor:0")
plate_boxes_tensor = plate_graph.get_tensor_by_name("detection_boxes:0")
plate_scores_tensor = plate_graph.get_tensor_by_name("detection_scores:0")

def detect_objects(sess, input_tensor, boxes_tensor, scores_tensor, image, threshold=0.5):
    img_expanded = np.expand_dims(image, axis=0)
    boxes, scores = sess.run([boxes_tensor, scores_tensor], feed_dict={input_tensor: img_expanded})
    boxes = np.squeeze(boxes)
    scores = np.squeeze(scores)
    detections = []
    h, w, _ = image.shape
    for i, score in enumerate(scores):
        if score >= threshold:
            ymin, xmin, ymax, xmax = boxes[i]
            x1 = int(xmin * w)
            y1 = int(ymin * h)
            x2 = int(xmax * w)
            y2 = int(ymax * h)
            detections.append((x1, y1, x2, y2, score))
    return detections

def blur_regions(image, detections, kernel_size=(251, 251)):
    processed_image = image.copy()
    for (x1, y1, x2, y2, score) in detections:
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(processed_image.shape[1], x2)
        y2 = min(processed_image.shape[0], y2)
        if x1 >= x2 or y1 >= y2:
            continue
        roi = processed_image[y1:y2, x1:x2]
        k_h = kernel_size[0] if kernel_size[0] % 2 != 0 else kernel_size[0] + 1
        k_w = kernel_size[1] if kernel_size[1] % 2 != 0 else kernel_size[1] + 1
        if roi.shape[0] < k_h or roi.shape[1] < k_w:
            continue
        for _ in range(3):
            roi = cv2.GaussianBlur(roi, (k_h, k_w), 0)
        processed_image[y1:y2, x1:x2] = roi
    return processed_image

def non_max_suppression(detections, iou_threshold=0.3):
    if not detections:
        return []
    boxes = []
    scores = []
    for (x1, y1, x2, y2, score) in detections:
        boxes.append([x1, y1, x2 - x1, y2 - y1])
        scores.append(score)
    indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=0.0, nms_threshold=iou_threshold)
    if len(indices) > 0:
        indices = indices.flatten()
        return [detections[i] for i in indices]
    return []

def process_and_save_image(input_path, output_path):
    image = cv2.imread(input_path)
    if image is None:
        return
    face_detections = detect_objects(face_sess, face_input_tensor, face_boxes_tensor, face_scores_tensor, image, FACE_THRESHOLD)
    plate_detections = detect_objects(plate_sess, plate_input_tensor, plate_boxes_tensor, plate_scores_tensor, image, PLATE_THRESHOLD)
    all_detections = face_detections + plate_detections
    filtered = non_max_suppression(all_detections)
    final = blur_regions(image, filtered, OBFUSCATION_KERNEL) if filtered else image
    cv2.imwrite(output_path, final, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])

# Criação de pastas
if not os.path.exists(OUTPUT_BASE_FOLDER):
    os.makedirs(OUTPUT_BASE_FOLDER)

# Registro de imagens já processadas
imagens_processadas = set()
if os.path.exists(PROCESSED_LOG):
    with open(PROCESSED_LOG, 'r') as f:
        imagens_processadas = set(l.strip() for l in f)

files_to_process = []
for root, dirs, files in os.walk(INPUT_BASE_FOLDER):
    rel_path = os.path.relpath(root, INPUT_BASE_FOLDER)
    output_dir = os.path.join(OUTPUT_BASE_FOLDER, rel_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for file in files:
        if file.lower().endswith(IMAGE_EXTENSIONS):
            input_file_path = os.path.join(root, file)
            output_file_path = os.path.join(output_dir, file)
            files_to_process.append((file, input_file_path, output_file_path))

# Loop principal com salvamento incremental
for file_name, input_path, output_path in tqdm(files_to_process, desc="Processing images"):
    if file_name in imagens_processadas:
        continue
    try:
        process_and_save_image(input_path, output_path)
        with open(PROCESSED_LOG, 'a') as f:
            f.write(file_name + '\n')
    except Exception as e:
        logging.error(f"Erro ao processar {file_name}: {e}")

# Encerramento de sessões
face_sess.close()
plate_sess.close()


2025-06-13 13:43:46.779925: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-13 13:43:46.783167: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-13 13:43:46.792163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749833026.807177  101714 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749833026.811693  101714 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749833026.823499  101714 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
Cell In[1], line 2
      1 import os
----> 2 import cv2
      3 import numpy as np
      4 import tensorflow as tf

ModuleNotFoundError: No module named 'cv2'

In [ ]:
!pip install opencv-python-headless


In [ ]:
!pip install opencv-python-headless --break-system-packages


In [ ]:
import sys
!{sys.executable} -m pip install --user opencv-python-headless


In [ ]:
import sys
!{sys.executable} -m pip install --break-system-packages opencv-python-headless


In [ ]:
!pip install tensorflow --break-system-packages


In [ ]:

!pip install tqdm


In [ ]:
pip install tqdm --break-system-packages


In [ ]:
source /home/katyperry/pyenv/bin/activate
pip install opencv-python-headless


In [ ]:
!{sys.executable} -m pip install opencv-python-headless


In [ ]:
!{sys.executable} -m pip install opencv-python-headless